# Learning on various text representations

## Preparing the dataset for FastText

Importing the necessary libraries

In [1]:
import json
import pandas as pd
from copy import deepcopy
import re
from tqdm import tqdm

In [2]:
# Import the file with additional text representations (only the paragraphs marked to be kept
# in the original corpus are included)

with open("/kaggle/input/ginco-with-additional-text-representations/Language-Processed-GINCO.json") as f:
    dataset = json.load(f)

dataset[0]

### Pre-processing dataset

Here we can create additional representations if we wish (see the notebook *2-Language-Processing-of-GINCO*).

1. Remove punctuation from each token

In [ ]:
        for instance in tqdm(dataset):
        text = instance["baseline_text"]
        
        # split text into tokens by white space
        token = text.split()
        
         
        # remove punctuation from each token
        table = str.maketrans('', '', punctuation)
        token = [word.translate(table) for word in token]

        # add a new key with punctuation removed
        instance["nopunctuation"] = " ".join(token)

2. Remove numbers from each token

3. Remove stopwords

4. Apply all (lowercase, remove punctuation, numbers, stopwords)

### Downcasting number of labels

In these experiments, we will not use all of the texts but only texts from 5 main categories, meaning that some categories will be merged into them, whereas some categories with a very small frequency will be discarded. Additionally, the texts marked us hard, will be discarded (see notebook *1-Preparing_Data_Hyperparameter_Search*).

We will start with a reduced set of labels (primary_level_3), then merge News and Opinionated News, and discard some of the lables.

In [3]:
# merge News and Opinionated News
for i in dataset:
    if i["primary_level_3"] == "Opinionated News" or i["primary_level_3"] == "News/Reporting":
        i["primary_level_3"] = "News"

Let's create train:test:dev split that contains only the wanted labels.

In [5]:
downcasted_labels = ['Information/Explanation', 'Promotion', 'News', 'Forum', 'Opinion/Argumentation']

train = [i for i in dataset if i["split"] == "train" and i["primary_level_3"] in downcasted_labels and not i["hard"]]
test = [i for i in dataset if i["split"] == "test" and i["primary_level_3"] in downcasted_labels and not i["hard"]]
dev = [i for i in dataset if i["split"] == "dev" and i["primary_level_3"] in downcasted_labels and not i["hard"]]

print("The train-dev-test splits consist of the following numbers of examples:", len(train), len(test), len(dev))

In [8]:
print(f"Number of all texts is {len(train)+len(test)+len(dev)}")

### Creating FastText texts

Now, let's create FastText train, dev and test files, containing the desired representation.

In [34]:
def fastText_files(representation):
    """
    This function creates and saves the test, train and dev file
    from the test, train and dev split of the dataset (named test, dev and train),
    using the "primary_level_3" level labels, and the chosen text representation.
    
    Possible representations: 'baseline_text', 'lemmas',
    'upos', 'xpos', 'ner', 'dependency', 'lowercase', 'lowercase_nopunctuation'
    
    The function returns a list of the following elements:
        - labels - which can be used for prediction and evaluation.
        - train file path
        - dev file path
        - test file path
    
    Args:
        representation (str): the name of the key (from the dataset)
                                of the text representation we want to use
    """
    # First create the dataframes from each split:
    
    train_df = pd.DataFrame(data=train, columns=[representation, "primary_level_3"])
    # Renaming columns to `text` and `labels`
    train_df.columns = ["text", "labels"]
    
    test_df = pd.DataFrame(data=test, columns=[representation, "primary_level_3"])
    test_df.columns = ["text", "labels"]
    
    dev_df = pd.DataFrame(data=dev, columns=[representation, "primary_level_3"])
    # Renaming columns to `text` and `labels`
    dev_df.columns = ["text", "labels"]
    
    print("The shape of the dataframes:")
    print(train_df.shape, dev_df.shape, test_df.shape)
    
    # Then create CSV files which FastText can read
    
    train_file_content=""

    for labels, text in train_df.loc[:, ["labels", "text"]].values:
        label = f"__label__{labels}"
        train_file_content += f"""{label} {text}\n"""
    
    train_path = ""
    train_path = representation + "-fasttext.train"

    with open(train_path,"w") as train_file:
        train_file.write(train_file_content)
    
    test_file_content=""
    
    for labels, text in test_df.loc[:, ["labels", "text"]].values:
        label = f"__label__{labels}"
        test_file_content += f"""{label} {text}\n"""
    
    test_path = ""
    test_path = representation + "-fasttext.test"
    
    with open(test_path,"w") as test_file:
        test_file.write(test_file_content)

    dev_file_content=""

    for labels, text in dev_df.loc[:, ["labels", "text"]].values:
        label = f"__label__{labels}"
        dev_file_content += f"""{label} {text}\n"""
    
    dev_path = ""
    dev_path = representation + "-fasttext.valid"

    with open(dev_path,"w") as dev_file:
        dev_file.write(dev_file_content)
        
    # Finally, create a list of labels which can be used for prediction and evaluation.
    # Let's inspect the labels:
    all_df_labels = train_df["labels"].unique().tolist()
    
    for i in test_df["labels"].unique().tolist():
        if i not in all_df_labels:
            all_df_labels.append(i)

    for i in dev_df["labels"].unique().tolist():
        if i not in all_df_labels:
            all_df_labels.append(i)

    print(f"Number of all labels: {len(all_df_labels)}")
    
    # Create a final list of labels in a FastText-appropriate format:
    LABELS = train_df.labels.unique().tolist()
    LABELS = [f"__label__{i}" for i in LABELS]
    
    return_list = [LABELS, train_path, dev_path, test_path]
    print(f"The function returned the following list: {return_list}")
    
    return return_list

In [56]:
lowercase_representation = fastText_files('upos')

In [57]:
# Define the label list:
LABELS = lowercase_representation[0]

LABELS

Inspect the created files:

In [58]:
train_file_path = lowercase_representation[1]

train_file = open(train_file_path,"r").read(2000)

train_file

In [59]:
test_file_path = lowercase_representation[3]

test_file = open(test_file_path,"r").read(2000)

test_file

In [60]:
dev_file_path = lowercase_representation[2]

dev_file = open(dev_file_path,"r").read(2000)

dev_file

# Train a fastText model

Importing the necessary libraries

In [61]:
!pip install parse

In [62]:
import fasttext as ft
import parse
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

## Input the data:

In [63]:
FT_train_file = train_file_path
FT_test_file = test_file_path
FT_dev_file = dev_file_path

## Training

In [64]:
def parse_test_file(path: str):
    """Reads fasttext formatted file and returns labels, texts."""
    with open(path, "r") as f:
        content = f.readlines()
    pattern = "{label} {text}\n"
    p = parse.compile(pattern)

    labels, texts = list(), list()
    for line in content:
        rez = p.parse(line)
        if rez is not None:
            labels.append(rez["label"])
            texts.append(rez["text"])
        else:
            print("error parsing line ", line)
    return labels, texts

def prediction_to_label(prediction):
    """Transforms predictions as returned by fasttext into pure labels."""
    return np.array(prediction[0])[:, 0]

def plot_cm(save=False, title=None):
    """
    Plots confusion matrix for prediction on the test set.
    Takes the predictions, named as y_pred, true values, named as y_true,
    and labels, named as LABELS.
    
    Arguments:
        save: whether the confusion matrix is saved. Defaults to False.
        title: the title of the confusion matrix. Defaults to None.
    """
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    classNames = LABELS
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    microF1 = f1_score(y_true, y_pred, labels=LABELS, average ="micro")
    macroF1 = f1_score(y_true, y_pred, labels=LABELS, average ="macro")

    print(f"{microF1:0.4}")
    print(f"{macroF1:0.4}")

    metrics = f"{microF1:0.4}, {macroF1:0.4}"
    if title:
        plt.title(title +";\n" + metrics)
    else:
        plt.title(metrics)
    plt.tight_layout()
    if save:
        plt.savefig(save)
    plt.show()
    return microF1, macroF1

In [65]:
results = list()

for i in range(5):
    model = ft.train_supervised(input=FT_train_file,
                                epoch = 350,
                                lr = 0.7,
                                wordNgrams=1,
                                verbose = 2
                                            )
    # Parse the test files so that labels and texts are separated
    y_true, y_texts = parse_test_file(FT_dev_file)

    # Evaluate te model on test data
    y_pred = model.predict(y_texts)
    y_pred = prediction_to_label(y_pred)

    # Plot the confusion matrix:
    m, M = plot_cm(save=False, title=f"Run: {i}")
    
    rezdict = dict(
        microF1=m,
        macroF1=M,
        run=i
    )
    results.append(rezdict)

In [66]:
results

In [67]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('_mpl-gallery')

# make data

x = []
mi = []
ma = []

for i in results:
    x.append(i['run'])
    mi.append(i['microF1'])
    ma.append(i["macroF1"])

# plot
fig, ax = plt.subplots(figsize=(6,3), dpi=100)

ax.plot(x, mi, linewidth=2.0, label="Micro F1")
ax.plot(x, ma, linewidth=2.0, label="Macro F1")

ax.set(xlim=(0,4.3),ylim=(0.5, 0.7),xticks=[0,1,2,3,4])

ax.set_xlabel('Runs')
ax.set_ylabel('F1 Score')
ax.legend(loc="lower right")

plt.tight_layout()
#plt.savefig("N-grams.png")
plt.show()

In [68]:
print(f"micro F1: {np.array(mi).mean():0.03} +/- {np.array(mi).std():0.02}")
print(f"macro F1: {np.array(ma).mean():0.03} +/- {np.array(ma).std():0.02}")